In [1]:
import pandas as pd

In [2]:
stocks = pd.read_csv('http://bit.ly/smallstocks')
stocks

,Date,Close,Volume,Symbol
0,2016-10-03,31.50,14070500,CSCO
1,2016-10-03,112.52,21701800,AAPL
2,2016-10-03,57.42,19189500,MSFT
3,2016-10-04,113.00,29736800,AAPL
4,2016-10-04,57.24,20085900,MSFT
5,2016-10-04,31.35,18460400,CSCO
6,2016-10-05,57.64,16726400,MSFT
7,2016-10-05,31.59,11808600,CSCO
8,2016-10-05,113.05,21453100,AAPL


In [3]:
stocks.index

RangeIndex(start=0, stop=9, step=1)

In [5]:
stocks.groupby('Symbol').Close.mean()

Symbol
AAPL    112.856667
CSCO     31.480000
MSFT     57.433333
Name: Close, dtype: float64

## Series with MultiIndex

In [7]:
ser = stocks.groupby(['Symbol', 'Date']).Close.mean()
# perform the groupby for each pair of the symbol and date
# the result is the series with a multi-index

In [9]:
ser.index
# the multi-index adds another dimension to our data
# now we have 2 things that we are indexing by

MultiIndex([('AAPL', '2016-10-03'),
            ('AAPL', '2016-10-04'),
            ('AAPL', '2016-10-05'),
            ('CSCO', '2016-10-03'),
            ('CSCO', '2016-10-04'),
            ('CSCO', '2016-10-05'),
            ('MSFT', '2016-10-03'),
            ('MSFT', '2016-10-04'),
            ('MSFT', '2016-10-05')],
           names=['Symbol', 'Date'])

In [10]:
ser.unstack()
# after unstacking our multi-index series becomes a dframe

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


In [17]:
df = stocks.pivot_table(values='Close', index='Symbol', columns='Date')
# the same things but done with pivoting
# the def agg function for the pivot_table is a "mean" but we can change that later


## Selection from Series with MultiIndex

In [13]:
ser.loc['AAPL']
# selecting from the multi-index series

Date
2016-10-03    112.52
2016-10-04    113.00
2016-10-05    113.05
Name: Close, dtype: float64

In [14]:
ser.loc['AAPL', '2016-10-03']

112.52

In [15]:
ser.loc[:, '2016-10-03']

Symbol
AAPL    112.52
CSCO     31.50
MSFT     57.42
Name: Close, dtype: float64

### Whether or not we work with a dframe or a series the selection is the same

### The reason is that they are the same things but with different shapes

In [19]:
df.loc[:, '2016-10-03']

Symbol
AAPL    112.52
CSCO     31.50
MSFT     57.42
Name: 2016-10-03, dtype: float64

## DataFrame with MultiIndex

In [21]:
stocks.set_index(['Symbol', 'Date'], inplace=True)
stocks

,,Close,Volume
Symbol,Date,,
CSCO,2016-10-03,31.50,14070500
AAPL,2016-10-03,112.52,21701800
MSFT,2016-10-03,57.42,19189500
AAPL,2016-10-04,113.00,29736800
MSFT,2016-10-04,57.24,20085900
CSCO,2016-10-04,31.35,18460400
MSFT,2016-10-05,57.64,16726400
CSCO,2016-10-05,31.59,11808600
AAPL,2016-10-05,113.05,21453100


In [22]:
stocks.index

MultiIndex([('CSCO', '2016-10-03'),
            ('AAPL', '2016-10-03'),
            ('MSFT', '2016-10-03'),
            ('AAPL', '2016-10-04'),
            ('MSFT', '2016-10-04'),
            ('CSCO', '2016-10-04'),
            ('MSFT', '2016-10-05'),
            ('CSCO', '2016-10-05'),
            ('AAPL', '2016-10-05')],
           names=['Symbol', 'Date'])

In [23]:
stocks.sort_index(inplace=True)

In [24]:
stocks
# the result is sorted by the outer index lvl and then by the inner

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
       2016-10-05  113.05  21453100
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
       2016-10-05   31.59  11808600
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900
       2016-10-05   57.64  16726400

## Selection from DataFrame with MultiIndex

In [50]:
stocks.loc['AAPL']

,Close,Volume
Date,,
2016-10-03,112.52,21701800
2016-10-04,113.00,29736800
2016-10-05,113.05,21453100


In [27]:
stocks.loc[('AAPL', '2016-10-03'), :]
# we are passing the tuple coz pandas may confuse the comma inside when there
# are no ()s

Close          112.52
Volume    21701800.00
Name: (AAPL, 2016-10-03), dtype: float64

In [31]:
# getting from multiple outer indecies
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), :]

,,Close,Volume
Symbol,Date,,
AAPL,2016-10-03,112.52,21701800
MSFT,2016-10-03,57.42,19189500


In [35]:
# getting from multiple inner indecies
stocks.loc[(slice(None), ['2016-10-03', '2016-10-04']), :]
# slice(None) -> getting info for all the outer indecies

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900

## Merging DataFrames with MultiIndexes

In [45]:
close = pd.read_csv('http://bit.ly/smallstocks', 
                    usecols=[0, 1, 3], 
                    index_col=['Symbol', 'Date']).sort_index()
close

Close
Symbol Date              
AAPL   2016-10-03  112.52
       2016-10-04  113.00
       2016-10-05  113.05
CSCO   2016-10-03   31.50
       2016-10-04   31.35
       2016-10-05   31.59
MSFT   2016-10-03   57.42
       2016-10-04   57.24
       2016-10-05   57.64

In [46]:
volume = pd.read_csv('http://bit.ly/smallstocks', 
                     usecols=[0, 2, 3], 
                     index_col=['Symbol', 'Date']).sort_index()
volume

Volume
Symbol Date                
AAPL   2016-10-03  21701800
       2016-10-04  29736800
       2016-10-05  21453100
CSCO   2016-10-03  14070500
       2016-10-04  18460400
       2016-10-05  11808600
MSFT   2016-10-03  19189500
       2016-10-04  20085900
       2016-10-05  16726400

## How to put 2 dframes together

In [48]:
both = pd.merge(close, volume, left_index=True, right_index=True)
# rather than defining a col to join on, we use left's and right's indecies
# we need to think about "how" param if we have missing val.s
# we need to have 2 dframes that have the same indecies and different col.s
both

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
       2016-10-05  113.05  21453100
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
       2016-10-05   31.59  11808600
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900
       2016-10-05   57.64  16726400

## Changing our gotten dframe into tidy data

### Tidy data rules:
    each variable must have its own column
    each observation must have its own row
    each value must have its own cell

In [49]:
both.reset_index()

,Symbol,Date,Close,Volume
0,AAPL,2016-10-03,112.52,21701800
1,AAPL,2016-10-04,113.00,29736800
2,AAPL,2016-10-05,113.05,21453100
3,CSCO,2016-10-03,31.50,14070500
4,CSCO,2016-10-04,31.35,18460400
5,CSCO,2016-10-05,31.59,11808600
6,MSFT,2016-10-03,57.42,19189500
7,MSFT,2016-10-04,57.24,20085900
8,MSFT,2016-10-05,57.64,16726400
